In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import *


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaReader") \
    .master("spark://localhost:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "users_created") \
    .option("startingOffsets", "earliest") \
    .load()


25/06/12 00:04:55 WARN Utils: Your hostname, MacBook-Pro-cua-Tam.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
25/06/12 00:04:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/tamdang/Desktop/Dev/Apache-Spark/spark-3.4.4-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tamdang/.ivy2/cache
The jars for the packages stored in: /Users/tamdang/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ea17bb22-f4a4-4185-842b-b29460819cf0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 498ms :: artifacts dl 12ms

In [ ]:
spark = SparkSession.builder \
    .appName("KafkaReader") \
    .master("spark://localhost:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

In [ ]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "users_created") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
df_parsed = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
query = df_parsed.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()
query.awaitTermination()

In [ ]:
spark.sparkContext.setLogLevel("WARN")



In [ ]:
df = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'broker:9092') \
    .option('subscribe', 'users_created') \
    .option('startingOffsets', 'earliest') \
    .load()


In [ ]:
schema = StructType([
    StructField("id", StringType(), False),
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), False),
    StructField("gender", StringType(), False),
    StructField("address", StringType(), False),
    StructField("post_code", StringType(), False),
    StructField("email", StringType(), False),
    StructField("username", StringType(), False),
    StructField("registered_date", StringType(), False),
    StructField("phone", StringType(), False),
    StructField("picture", StringType(), False)
])

In [ ]:
sel = df.selectExpr("CAST(value AS STRING)") \
        .select(from_json(col('value'), schema).alias('data')).select("data.*")

In [ ]:
query = sel.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="5 seconds") \
    .start()
query.awaitTermination()